First cell is necessary for running this notebook with django env (as if it was executed from an API call).

In [ ]:
import os, sys
DJANGO_ROOT = os.path.abspath(os.path.join(os.getcwd(),'../../'))
sys.path.insert(0,DJANGO_ROOT)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
import django
django.setup()

In [ ]:
from app.models import Profile, Image, LikedImages
import pickle as pckl

# Like image

In [ ]:
data = {'like': False, 'id':1 }

In [ ]:
user = User.objects.all().first()

In [ ]:
profile = Profile.objects.all().first()

In [ ]:
# import numpy as np
# profile.pref_vec = pckl.dumps( np.zeros_like(pckl.loads(str(profile.pref_vec))))
# profile.img_count = 0
# profile.save()

In [ ]:
query_res = LikedImages.objects.filter(user= user.id, img=data['id'] )

if data['like']:
    # Add the image to list if it's not there yet
    if len(query_res) is 0:
        image = Image.objects.filter(id=data['id']).first()

        # update user preferences
        profile = Profile.objects.filter(user_id=user.id).first()
        profile_vec = pckl.loads(str(profile.pref_vec))
        img_vec     = pckl.loads(str(image.txt_vec))

        profile_vec = ((profile_vec * profile.img_count) + img_vec) / (profile.img_count + 1 )
        profile.img_count += 1
        profile.pref_vec = pckl.dumps(profile_vec)
        profile.save()

        # save like
        li = LikedImages(user=user, img=image)
        li.save()
else:
    if len(query_res) is 1:
        result = query_res.first()
        
        # update user preferences
        profile = Profile.objects.filter(user_id=user.id).first()
        profile_vec = pckl.loads(str(profile.pref_vec))
        img_vec     = pckl.loads(str(result.img.txt_vec))

        profile_vec = ((profile_vec * profile.img_count) - img_vec)
        profile_vec = profile_vec if profile.img_count == 1 else profile_vec / (profile.img_count - 1 )
        profile.img_count -= 1
        profile.pref_vec = pckl.dumps(profile_vec)
        profile.save()
        
        result.delete()

In [ ]:
len(LikedImages.objects.all())

Test for succ user update:

In [ ]:
user = Profile.objects.all().first()

In [ ]:
pckl.loads(str(user.pref_vec))

In [ ]:
Profile.objects.first().img_count

# Retrieve recommendations

In [ ]:
from django.contrib.auth.models import User

In [ ]:
images = Image.objects.all()

In [ ]:
user = User.objects.all().first()
profile = Profile.objects.filter(user_id = user.id).first()

In [ ]:
pref_vec = pckl.loads(str(profile.pref_vec))

In [ ]:
sim = []
for img in images:
    img_vec = pckl.loads(str(img.txt_vec)) 
    similarity = np.arccos(np.dot(img_vec,pref_vec) / (np.linalg.norm(img_vec) * np.linalg.norm(pref_vec))) 
    sim.append( (img.id,similarity))

In [ ]:
sim.sort(key=lambda tup: tup[1])

In [ ]:
top_k = 16
indices = map(lambda rec: rec[0], sim[:top_k])

In [ ]:
indices

In [ ]:
Image.objects.filter(id__in = indices)

# Retrieve recommendations (proper way)

In [13]:
from django.contrib.auth.models import User
from app.models import Image
from project.project_config import empty_embed_vec, obj2pickled , pickled2obj, GALLERY_IMG_NUM
from tqdm import tqdm
import numpy as np

In [14]:
images = Image.objects.all()

In [15]:
user = User.objects.all().first()
profile = Profile.objects.filter(user_id = user.id).first()

In [16]:
#user = self.request.user
images = Image.objects.all().exclude(id__in=LikedImages.objects.filter(user=user).values_list('img', flat=True))
liked_images = Image.objects.all().filter(id__in=LikedImages.objects.filter(user=user).values_list('img', flat=True))

In [17]:
total_liked_count = len(liked_images)

In [ ]:
# From liked images extract the number of classes and mean vec for each label
usr_class_emb = {}
for image in liked_images:
    if image.label not in usr_class_emb:
        class_emb = {'vec':empty_embed_vec(), 'count':0}
    else:
        class_emb = usr_class_emb[image.label]
    class_emb['vec'] += pickled2obj(image.txt_vec)
    class_emb['count'] +=1
    usr_class_emb[image.label] = class_emb
    
for _,val in usr_class_emb.items():
    val['vec'] /= val['count']

In [ ]:
def find_similar(pref_vec,images=None, k_similar=20):
    if images is None:
        images = Image.objects.all().exclude(id__in=LikedImages.objects.filter(user=user).values_list('img', flat=True))
    sim = []
    for img in images:
        img_vec = pickled2obj(img.txt_vec)
        similarity = np.arccos(np.dot(img_vec, pref_vec) / (np.linalg.norm(img_vec) * np.linalg.norm(pref_vec)))
        sim.append((img.id, similarity))

    sim.sort(key=lambda tup: tup[1])
    indices = map(lambda rec: rec[0], sim[:k_similar])
    return indices

In [ ]:
# Fetch k similar for each class and then
output_images_indices = []
class_count = len(usr_class_emb)
for val in usr_class_emb.values():
    class_count -= 1
    pref_vec = val['vec']
    sim_img_count = round(val['count'] / total_liked_count * GALLERY_IMG_NUM) 
    if class_count is 0:
        sim_img_count = GALLERY_IMG_NUM - len(output_images)
    sim_image_indices = find_similar(pref_vec, images, sim_img_count)
    output_images_indices += sim_image_indices

In [ ]:
Image.objects.filter(id__in = output_images_indices)